In [1]:
import numpy as np
import cv2 as cv
from motrackers import IOUTracker
from motrackers.utils import select_caffemodel, select_videofile

### Setup 

#### Choose file paths


In [2]:
# video_file = select_videofile('..')
# prototxt, weights = select_caffemodel('..')
# display(video_file, prototxt, weights)

video_file = "/home/adi/git_clones/multi-object-tracker/video_data/people.mp4"
prototxt = "/home/adi/git_clones/multi-object-tracker/pretrained_models/caffemodel_weights/MobileNetSSD_deploy.prototxt" 
weights = "/home/adi/git_clones/multi-object-tracker/pretrained_models/caffemodel_weights/MobileNetSSD_deploy.caffemodel"

#### Set hyper parameters and constants

In [3]:
print_tracks = False   # print the tracker output if True

video = video_file

model = {"prototxt": prototxt,
         "weights": weights,
         "object_names": {0: 'background', 
                          1: 'aeroplane', 
                          2: 'bicycle', 
                          3: 'bird',
                          4: 'boat',
                          5: 'bottle',
                          6: 'bus', 
                          7: 'car', 
                          8: 'cat', 
                          9: 'chair',
                          10: 'cow', 
                          11: 'diningtable', 
                          12: 'dog', 
                          13: 'horse',
                          14: 'motorbike', 
                          15: 'person', 
                          16: 'pottedplant',
                          17: 'sheep', 
                          18: 'sofa', 
                          19: 'train',
                          20: 'tvmonitor'},
         "threshold": 0.2,
         "confidence_threshold": 0.2,
         "pixel_std":1/127.5,
         "pixel_mean": 127.5,
         "input_size": (300, 300)
        }

max_object_lost_count = 5   # maximum number of object losts counted when the object is being tracked

np.random.seed(12345)
bbox_colors = {key: np.random.randint(0, 255, size=(3,)).tolist() for key in model['object_names'].keys()}

### Initialize
* Initialize video
* Initialize network
* Initialize tracker

In [4]:
cap = cv.VideoCapture(video)
net = cv.dnn.readNetFromCaffe(model["prototxt"], model["weights"])

In [5]:
tracker = IOUTracker(max_lost=2, 
                     iou_threshold=0.5, 
                     min_detection_confidence=0.7,
                     max_detection_confidence=0.9)

In [6]:
(H, W) = (None, None)
writer = None

while True:
    ok, image = cap.read()
    
    if not ok:
        print("Cannot read the video feed.")
        break
    
    if W is None or H is None: 
        (H, W) = image.shape[:2]
    
    image_resized = cv.resize(image, model["input_size"])

    blob = cv.dnn.blobFromImage(
        image_resized, model["pixel_std"], model["input_size"], 
        (model["pixel_mean"], model["pixel_mean"], model["pixel_mean"]), False
    )
    net.setInput(blob)
    detections = net.forward()

    rows = image_resized.shape[0]
    cols = image_resized.shape[1]
    
    boxes, confidences, classIDs, detections_bbox = [], [], [], []

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > model['confidence_threshold']:
            class_id = int(detections[0, 0, i, 1])

            # object location 
            left = int(detections[0, 0, i, 3] * cols) 
            top = int(detections[0, 0, i, 4] * rows)
            right = int(detections[0, 0, i, 5] * cols)
            bottom = int(detections[0, 0, i, 6] * rows)
            
            # scaling factor of image
            height_factor = image.shape[0]/float(model["input_size"][0])
            width_factor = image.shape[1]/float(model["input_size"][1])
            
            # scale object detection bounding box to original image
            left = int(width_factor * left) 
            top = int(height_factor * top)
            
            right = int(width_factor * right)
            bottom = int(height_factor * bottom)
            
            width = right - left
            height = bottom - top
            
            boxes.append([left, top, width, height])
            confidences.append(float(confidence))
            classIDs.append(int(class_id))
        
    indices = cv.dnn.NMSBoxes(boxes, confidences, model["confidence_threshold"], model["threshold"])
    
    class_ids_to_track = []
    confidences_track = []
    if len(indices)>0:
        for i in indices.flatten():
            x, y, w, h = boxes[i][0], boxes[i][1], boxes[i][2], boxes[i][3]
            
            detections_bbox.append((x, y, x+w, y+h))
            class_ids_to_track.append(classIDs[i])
            
            clr = [int(c) for c in bbox_colors[classIDs[i]]]
            cv.rectangle(image, (x, y), (x+w, y+h), clr, 2)
            
            confidences_track.append(confidences[i])
            label = "{}:{:.4f}".format(model["object_names"][classIDs[i]], confidences[i])
            
            (label_width, label_height), baseLine = cv.getTextSize(
                label, cv.FONT_HERSHEY_SIMPLEX, 0.5, 2)
            
            y_label = max(y, label_height)
            
            cv.rectangle(image, 
                         (x, y_label-label_height),
                         (x+label_width, y_label+baseLine), 
                         (255, 255, 255), 
                         cv.FILLED)
            
            cv.putText(image, label, (x, y_label), cv.FONT_HERSHEY_SIMPLEX, 0.5, clr, 2)
        
    tracks = tracker.update(detections_bbox, class_ids_to_track, confidences_track)
    
    if print_tracks:
        print("Tracker Outputs:\n", tracks, "\n")
    
    for track in tracks:
        track_count, trackid, class_id, centroid, track_bbox, track_info = track
        
        text = "ID {}".format(trackid)
        
        cv.putText(image, 
                   text, 
                   (centroid[0] - 10, centroid[1] - 10),
                   cv.FONT_HERSHEY_SIMPLEX,
                   0.5, 
                   (0, 255, 0), 
                   2)
        
        cv.circle(image, 
                  (centroid[0], centroid[1]), 
                  4, 
                  (0, 255, 0), 
                  -1)
    
    cv.imshow("image", image)
    
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
        
    if writer is None:
        fourcc = cv.VideoWriter_fourcc(*"MJPG")
        writer = cv.VideoWriter("output.avi", fourcc, 30, (W, H), True)
    writer.write(image)

writer.release()
cap.release()
cv.destroyWindow("image")

Cannot read the video feed.
